# Paper reproduction: Communication-Efficient Distributed SGD using Preamble-based Random Access by Choi Available at https://arxiv.org/abs/2105.09427

Straighfoward reproduction without classes

## Imports

In [1]:
# Import needed libraries
import numpy as np # arrays
import matplotlib.pyplot as plt #ploting
import scipy.optimize as sciopt    # linprog
from scipy.optimize import nnls # NNLS SOLVER

## Auxiliary functions

Construct the codebook as a scaled cross polytope:
$$ \cal{C}_{cp} = \{ \pm \textrm{R}e_l : \{1,...,\textrm{L}\}\} $$
with $ R = \sqrt{L} $, with $L$ being the block length.

In [3]:
def construct_codebook(blockLength):
    # Construct the codebook as a scaled cross polytope
    R = np.sqrt(blockLength)
    codebook = np.concatenate(((np.eye(blockLength)*R),(np.eye(blockLength)*(-R))),axis=0) + 0 # add zero to fix -0.0 issue
    return codebook

In [4]:
def quantize(codebook,numBlocks,blockLength,gradient,Vmax):
        
        codebook = codebook
        
        prob = np.zeros(numBlocks)
        norms = np.zeros(numBlocks)
        c_idx = np.zeros(numBlocks, int)
        
        for block in range(numBlocks):
            
            begin = block*blockLength
            end = block*blockLength+blockLength
            
            v_d = gradient[begin:end]
            v_til = (v_d)/np.linalg.norm(v_d)
            norms[block] = np.linalg.norm(v_d)
            
            if(norms[block] > Vmax):
                
                v_d = v_d*(Vmax/norms[block])
                norms[block] = Vmax
                gradient[begin:end] = v_d

            # solving the linear system
            M = codebook.shape[0]
            A = np.concatenate((codebook.T,np.ones((1,M))),axis=0)
            b = np.concatenate((v_til,np.ones(1)),axis=0)
            
            
            a_vec = nnls(A, b)[0] # Using Non-negative Least Squares
            ## scipy.optimize.nnls solves the KKT (Karush-Kuhn-Tucker) conditions for the non-negative least squares problem.

            # print(np.sum(np.abs(v_til-a_vec@codebook)**2))
            # input()
            
            
            c_idx[block] = np.random.choice(M, p=a_vec) # use a_vec as probabilities for choosing the codeword
            # print(c_idx[block])
            # a_probs[block] = a_vec[c_idx[block]]
            prob[block] = norms[block]/Vmax
            
        return prob, norms, c_idx, gradient # returns the norms and the indexs of the codewords

In [5]:
def RA_mix(prob, qidx, codebook,blockLength, numBlocks):
    
    grad = np.zeros(L)
    for k in range(K):
        
        for block in range(numBlocks):
            
            begin = block*blockLength
            end = block*blockLength+blockLength
            p=0
            if np.random.normal() < prob[k,block]:
                p = 1
            qi = qidx[k,block]
            grad[begin:end] += (p+np.random.normal())*codebook[qi]
            
    
    grad = grad/K
    
    return grad

In [ ]:
def RA_mix(prob, qidx, codebook,blockLength, numBlocks):
    
    
    noise 
    grad = np.zeros(L)
    for k in range(K):
        
        for block in range(numBlocks):
            
            begin = block*blockLength
            end = block*blockLength+blockLength
            p=0
            if np.random.normal() < prob[k,block]:
                p = 1
            qi = qidx[k,block]
            grad[begin:end] += (p+np.random.normal())*codebook[qi]
            
    
    grad = grad/K
    
    return grad

## Definitions

In [7]:
# Definitions
K = 10 # Number of datasets distributed over K devices (i.e., 1 data set per device)
numBlocks = 10  # amount of blocks in a gradient vector (D subvectors)
blockLength = 8 # block length
L = blockLength*numBlocks # Lenght of w
u =  0.01 # step size or learning rate
T = 100 # Number of iterations
Vmax = np.sqrt(blockLength)

## Testing

In [17]:
# %%time
mcr = 100
mse = 0
snr = 10
N = 100 # Number of antennas
codebook = construct_codebook(blockLength)

for mx in range(mcr):
    true_g = np.zeros(L)
    est_g = np.zeros(L)
    gradients = np.random.normal(size = (K,L))
    probs = np.zeros((K,numBlocks))
    qidx = np.zeros((K,numBlocks),int)
    norms_v = np.zeros((K,numBlocks))

    for k in range(K):
        probs[k,:], norms_v[k,:], qidx[k,:], gradients[k,:] = quantize(codebook,numBlocks,blockLength,gradients[k,:],Vmax)

    est_g = RA_mix(probs, qidx, codebook,blockLength, numBlocks)
    true_g = np.sum(gradients,axis = 0)/K

    mse +=  np.sum(np.abs(true_g-est_g)**2)/mcr
    
print(mse)

17.28226930468923
